<center><h1> Artifical and Computational Intelligence Assignment 1 </h1></center>
 
Problem Solving by using A*

Weightage: 13%

Date of Posting - 20/July/2020

Deadline - 30/July/2020

Please Enter group and Individual information below:

Group ID/Number:

Group Members BITS ID:

<h3>Things to Follow</h3>

1. Use appropriate data structures for the fringes and explain the reasoning behind the usage. You need not create data structures, instead use available libraries directly.

2. Avoid any hard-coding unless absolutely necessary.

3. Provide proper documentation

4. Read the question file provided carefully and attempt all the questions! 

Coding begins now!!!

1) Define the environment representation in the following code block

In [ ]:
#Environment representation goes here



Define the haversine formula and what it does

# Below the Huristic Distance is calculated by Directly calculating the distance between two Geo-Location that is Lattitude and Longitude

In [31]:
#Function for calculating distance from each node/city to Destination
import geopy.distance
def calcDist(coords_1,coords_2):
    return geopy.distance.geodesic(coords_1, coords_2).km
    


In [14]:
class Node:
    # Initialize the class
    def __init__(self, name:str, parent:str):
        self.name = name
        self.parent = parent
        self.g = 0 # Distance to start node
        self.h = 0 # Huristic Distance to goal node
        self.f = 0 # Total cost (f = g+h)
    # Compare nodes
    def __eq__(self, other):
        return self.name == other.name
    # Sort nodes
    def __lt__(self, other):
         return self.f < other.f
    # Print node
    def __repr__(self):
        return ('({0},{1})'.format(self.name, self.f))


In [15]:
class Graph:
    # Initialize the class
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()
    # Create an undirected graph by adding symmetric edges
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist
    # Add a link from A and B of given distance, and also add the inverse link if the graph is undirected
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance
    # Get neighbors or a neighbor
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)
    # Return a list of nodes in the graph
    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)

3) Implementation of A* Algorithm . Feel free to add code blocks for each methods needed starting here.
Please modularize the implementation of A* and write each of them in a code block. 

In [24]:
def astar_search(graph, heuristics, start, end):
    
    # Create lists for open nodes and closed nodes
    open = []
    closed = []
    # Create a start node and an goal node
    start_node = Node(start, None)
    goal_node = Node(end, None)
    # Add the start node
    open.append(start_node)
    
    # Loop until the open list is empty
    while len(open) > 0:
        # Sort the open list to get the node with the lowest cost first
        open.sort()
        # Get the node with the lowest cost
        current_node = open.pop(0)
        # Add the current node to the closed list
        closed.append(current_node)
        
        # Check if we have reached the goal, return the path
        if current_node == goal_node:
            path = []
            while current_node != start_node:
                path.append(current_node.name + ': ' + str(current_node.g))
                current_node = current_node.parent
            path.append(start_node.name + ': ' + str(start_node.g))
            # Return reversed path
            return path[::-1]
        # Get neighbours
        neighbors = graph.get(current_node.name)
        # Loop neighbors
        for key, value in neighbors.items():
            # Create a neighbor node
            neighbor = Node(key, current_node)
            # Check if the neighbor is in the closed list
            if(neighbor in closed):
                continue
            # Calculate full path cost
            neighbor.g = current_node.g + graph.get(current_node.name, neighbor.name)
            neighbor.h = heuristics.get(neighbor.name)
            neighbor.f = neighbor.g + neighbor.h
            # Check if neighbor is in open list and if it has a lower f value
            if(add_to_open(open, neighbor) == True):
                # Everything is green, add neighbor to open list
                open.append(neighbor)
    # Return None, no path is found
    return None
# Check if a neighbor should be added to open list
def add_to_open(open, neighbor):
    for node in open:
        if (neighbor == node and neighbor.f > node.f):
            return False
    return True

In [17]:
#Code Block 2 .. 

Call your main function/algorithm block in the next code block with appropriate input representation

In [35]:
def main():
    # Create a graph
    routeMaze = Graph()
    # Create graph connections (Actual distance)
    routeMaze.connect('Bengaluru', 'Sirsi', 405)
    routeMaze.connect('Bengaluru', 'Hubli', 412)
    routeMaze.connect('Bengaluru', 'Belgum', 507)
    routeMaze.connect('Sirsi', 'Ankola', 91)
    routeMaze.connect('Hubli', 'Yellapur', 71)
    routeMaze.connect('Hubli', 'Dharwad', 18)
    routeMaze.connect('Belgum', 'Chorlaghat', 59)
    routeMaze.connect('Yellapur', 'Ankola', 72)
    routeMaze.connect('Dharwad', 'Goa', 140)
    routeMaze.connect('Chorlaghat', 'Goa', 67)
    routeMaze.connect('Ankola', 'Goa', 111)
    # Make graph undirected, create symmetric connections
    routeMaze.make_undirected()
    
    #Loding the Geo Location in a Dict
    location = {}
    location['Belgum'] = (15.8497,74.4977)
    location['Chorlaghat'] =(15.6496,74.1189)
    location['Hubli'] = (15.3647,75.1240)
    location['Dharwad'] = (15.4589,75.0078)
    location['Yellapur'] = (14.9643,74.7121)
    location['Sirsi'] = (14.6196,74.8441)
    location['Ankola'] = (14.6653,74.3001)
    location['Bengaluru'] = (12.9716,77.5946)
    location['Goa'] = (15.2993,74.1240)
    
    #Calculating Huristic Val for each Node
    
    heuristics = {}
    heuristics['Belgum'] = calcDist(location['Belgum'],location['Goa'])
    heuristics['Chorlaghat'] =calcDist(location['Chorlaghat'],location['Goa'])
    heuristics['Hubli'] = calcDist(location['Hubli'],location['Goa'])
    heuristics['Dharwad'] = calcDist(location['Dharwad'],location['Goa'])
    heuristics['Yellapur'] = calcDist(location['Yellapur'],location['Goa'])
    heuristics['Sirsi'] = calcDist(location['Sirsi'],location['Goa'])
    heuristics['Ankola'] = calcDist(location['Ankola'],location['Goa'])
    heuristics['Bengaluru'] =calcDist(location['Bengaluru'],location['Goa'])
    heuristics['Goa'] = calcDist(location['Goa'],location['Goa'])
    
    path = astar_search(routeMaze, heuristics, 'Bengaluru', 'Goa')
    print(path)
    print()    
    


The agent should provide expected output for questions mentioned below in the subsequent blocks

(3.1) Path taken to reach destination from Bengaluru

In [36]:
# Execute statement to retrieve the path taken here
if __name__ == "__main__": main()

['Bengaluru: 0', 'Hubli: 412', 'Dharwad: 430', 'Goa: 570']



(3.2) Cost of the path

In [ ]:
# Execute statement to retrieve the cost of the path here


(3.3) Total Number of nodes vistied to get this state

In [ ]:
# Execute statement to retrieve the total number of nodes visited to get this state here


<center>All the best!! Happy Coding!!Let human intelligence prevail</center>
<center>**********************************************************</center>